In [68]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

df = pd.read_csv('data/table_matches_new.csv')
df = df.drop(columns=['Unnamed: 0'])
count = 6865 
while count != 7122 :
    count += 1
    if (count % 2) == 0:
        sleep(1)
    sleep(1)
    url = f"https://soccer365.ru/games/173{count}/"

    response = requests.get(url)

    soup = BeautifulSoup(response.text,'lxml')

    title = ["общая информация","Дата",'Команда1',"счет1",'Команда2',"счет2","xG1","xG2","Удары1","Удары2",'Удары в створ1','Удары в створ2','Сейвы1','Сейвы2','Владение %1','Владение %2','Угловые1','Угловые2','Нарушения1','Нарушения2','Офсайды1','Офсайды2','Желтые карточки1','Желтые карточки2','Красные карточки1','Красные карточки2','Атаки1','Атаки2','Опасные атаки1','Опасные атаки2','Передачи1','Передачи2','Точность передач %1','Точность передач %2','Штрафные удары1','Штрафные удары2','Вбрасывания1','Вбрасывания2','Навесы1','Навесы2','Температура',"Признак погоды",'дата ПМ1','Пголы1',"Пза матч1","Ппобед1","Пничьи1","Ппорож1",'дата ПМ2','Пголы2',"Пза матч2","Ппобед2","Пничьи2","Ппорож2","поб1","X","поб2","ТМ 2.5","ТБ 2.5"]
    lister = list()

    #Общая инфа и время матча
    data = soup.find("div",class_="block_header bkcenter").text.replace("\n","")
    date_list = data.split()
    date_time = date_list[-2] + ' ' + date_list[-1]
    lister.append(data)
    lister.append(date_time)

    #Название команд и счет

    result_match_1= soup.find("div",class_="live_game left")
    result_match_2= soup.find("div",class_="live_game right")

    name_1_com = result_match_1.find("div",class_="live_game_ht").text.replace("\n","")
    goal_1_com = result_match_1.find("div",class_="live_game_goal").text.replace("\n","")

    name_2_com = result_match_2.find("div",class_="live_game_at").text.replace("\n","")
    goal_2_com = result_match_2.find("div",class_="live_game_goal").text.replace("\n","")
    lister.append(name_1_com)
    lister.append(goal_1_com)
    lister.append(name_2_com)
    lister.append(goal_2_com)

    #основные  данные о матче
    gm_block= soup.find("div",class_="block_body")

    result_matches_1 = gm_block.find_all("div",class_="stats_items")[0]
    result_table_matches_1 = result_matches_1.find_all("div", class_="stats_inf")
    for i in (result_table_matches_1):
        lister.append(float(i.text))


    result_matches_2 = gm_block.find_all("div",class_="stats_items")[1]
    result_table_matches_2 = result_matches_2.find_all("div", class_="stats_inf")
    if len(result_table_matches_2) == 16:
        for j in (result_table_matches_2):
            lister.append(float(j.text))
    else :
        result_table_matches_2 = result_table_matches_2[0:16]
        for j in (result_table_matches_2):
            lister.append(float(j.text))
        
    #Погода
    preview_st = soup.find("div",class_="preview_item st")
    temper = preview_st.find("span",class_="red").text

    priznac_pog = preview_st.find_all("span",class_="min_gray")[1].text
    lister.append(temper)
    lister.append(priznac_pog)

    #инфа за прошлые 5 матчей
    preview_st = soup.find("div",class_="gm_block inline-block")
    home_prewie = preview_st.find('div',class_='live_block_hf')
    home_table = home_prewie.find('table',class_="game_form_team_stats")

    home_old_match = home_prewie.find('span', class_="size10")
    lister.append(home_old_match.text)

    home_table_text = home_table.find_all('td')[1:6]
    for i in (home_table_text):
        lister.append(float(i.text))



    atac_prewie = preview_st.find('div',class_='live_block_hf right')
    atac_table = atac_prewie.find('table',class_="game_form_team_stats")

    atac_old_match = atac_prewie.find('span', class_="size10")
    lister.append(atac_old_match.text)

    atac_table_text = atac_table.find_all('td')[1:6]
    for j in (atac_table_text):
        lister.append(float(j.text))
        
    #Коэфиценты
    coeficent_all = soup.find_all("div",class_="block_body_nopadding")[3]
    coficient = coeficent_all.find_all('div',class_='odds_item odds_logo')

    def cof_func(coficient):
        list_coef = list()
        for i in coficient:
            odds_coeff = i.find_all('div',class_='odds_coeff')
            for j in odds_coeff:
                if j == '\n':
                    continue
                if j == '\xa0':
                    list_coef.clear()
                    break
                list_coef.append(float(j.text))
            if len(list_coef) == 5:
                return  list_coef  
            
    coficient_result = cof_func(coficient)
    lister.extend(coficient_result)

    df.loc[len(df)] = lister
df

ValueError: cannot set a row with mismatched columns

In [69]:
count

6933

In [70]:
df['счет1'].value_counts()
df['счет2'].value_counts()

1    24
0    19
2    13
3     8
4     2
6     1
5     1
Name: счет2, dtype: int64

In [71]:
df['Признак погоды'].value_counts()

солнечно                 64
переменная облачность     3
ясно                      1
Name: Признак погоды, dtype: int64

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 59 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   общая информация     68 non-null     object 
 1   Дата                 68 non-null     object 
 2   Команда1             68 non-null     object 
 3   счет1                68 non-null     object 
 4   Команда2             68 non-null     object 
 5   счет2                68 non-null     object 
 6   xG1                  68 non-null     float64
 7   xG2                  68 non-null     float64
 8   Удары1               68 non-null     float64
 9   Удары2               68 non-null     float64
 10  Удары в створ1       68 non-null     float64
 11  Удары в створ2       68 non-null     float64
 12  Сейвы1               68 non-null     float64
 13  Сейвы2               68 non-null     float64
 14  Владение %1          68 non-null     float64
 15  Владение %2          68 non-null     float

In [73]:
#df.to_csv('data/table_matches40.csv')